In [ ]:
pip install pandas_datareader ipywidgets uszipcode python-Levenshtein

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
import os

import matplotlib
from uszipcode import SearchEngine

import pandas_datareader as web   
from pandas_datareader import wb
from ipywidgets import Dropdown
from ipywidgets import interact
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import Normalize
import datetime
import warnings
warnings.filterwarnings("ignore")

from shapely.geometry import Polygon, MultiPolygon


In [2]:
## get state fips from zipcode
def get_state_fips(zipcode):
    search = SearchEngine()
    zipcode_info = search.by_zipcode(zipcode)

    state_abbr = zipcode_info.state

    state_to_fips = {
        'AK': '02', 'AL': '01', 'AR': '05', 'AS': '60', 'AZ': '04', 'CA': '06', 'CO': '08', 'CT': '09', 
        'DC': '11', 'DE': '10', 'FL': '12', 'GA': '13', 'GU': '66', 'HI': '15', 'IA': '19', 'ID': '16', 
        'IL': '17', 'IN': '18', 'KS': '20', 'KY': '21', 'LA': '22', 'MA': '25', 'MD': '24', 'ME': '23', 
        'MI': '26', 'MN': '27', 'MO': '29', 'MS': '28', 'MT': '30', 'NC': '37', 'ND': '38', 'NE': '31', 
        'NH': '33', 'NJ': '34', 'NM': '35', 'NV': '32', 'NY': '36', 'OH': '39', 'OK': '40', 'OR': '41', 
        'PA': '42', 'PR': '72', 'RI': '44', 'SC': '45', 'SD': '46', 'TN': '47', 'TX': '48', 'UT': '49', 
        'VA': '51', 'VI': '78', 'VT': '50', 'WA': '53', 'WI': '55', 'WV': '54', 'WY': '56'
    }

    state_fips = state_to_fips.get(state_abbr)

    return state_fips

In [3]:
all_data = pd.read_csv("ALL_processed_Change.csv")

In [4]:
all_data['Zip Code'].nunique()

391

In [5]:
all_data['zip'] = all_data['Zip Code'].astype(str).str.zfill(5)
all_data['state'] = all_data['zip'].apply(get_state_fips)
fips_to_msa = {
    '06': 'SF',
    '25': 'Boston',
    '33': 'Boston',
    '13': 'Atlanta',
    '48': 'Dallas'
}
# create a new column using the map function
all_data['msa'] = all_data['state'].map(fips_to_msa)

In [6]:
all_data

,y_dt,Zip Code,Population Change (%),Housing Change (%),Housing/person Change (%),rent_change,zip,state,msa
0,2011-07-01,75001,0.060002,0.049557,-0.011414,0.055810,75001,48,Dallas
1,2011-10-01,75001,0.060002,0.049557,-0.011200,0.065749,75001,48,Dallas
2,2012-01-01,75001,0.060155,0.049682,-0.011021,0.108097,75001,48,Dallas
3,2012-04-01,75001,0.053480,0.048808,-0.005096,0.050549,75001,48,Dallas
4,2012-07-01,75001,0.047034,0.048059,0.000669,-0.006383,75001,48,Dallas
...,...,...,...,...,...,...,...,...,...
13112,2018-10-01,3079,0.005065,0.012164,0.007165,0.201314,03079,33,Boston
13113,2019-01-01,3079,0.005135,0.007970,0.002874,0.197204,03079,33,Boston
13114,2019-04-01,3079,0.005100,0.003691,-0.001364,0.218893,03079,33,Boston
13115,2019-07-01,3079,0.005135,-0.000671,-0.005717,0.216178,03079,33,Boston


In [7]:
zip_shp = geopandas.read_file('us_zip/tl_rd22_us_zcta520.shp')
zip_shp['ZIP'] = zip_shp['ZCTA5CE20'].astype(int)
zip_shp

,ZCTA5CE20,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry,ZIP
0,35592,35592,B5,G6350,S,298552385,235989,+33.7427261,-088.0973903,"POLYGON ((-88.24735 33.65390, -88.24713 33.654...",35592
1,35616,35616,B5,G6350,S,559506992,41870756,+34.7395036,-088.0193814,"POLYGON ((-88.13997 34.58184, -88.13995 34.582...",35616
2,35621,35621,B5,G6350,S,117838488,409438,+34.3350314,-086.7270557,"POLYGON ((-86.81659 34.34960, -86.81649 34.349...",35621
3,35651,35651,B5,G6350,S,104521045,574316,+34.4609087,-087.4801507,"POLYGON ((-87.53087 34.42492, -87.53082 34.429...",35651
4,36010,36010,B5,G6350,S,335675180,236811,+31.6598950,-085.8128958,"POLYGON ((-85.95712 31.67744, -85.95676 31.677...",36010
...,...,...,...,...,...,...,...,...,...,...,...
33786,10540,10540,B5,G6350,S,1394970,27777,+41.3259819,-073.7277119,"POLYGON ((-73.73580 41.33577, -73.73501 41.335...",10540
33787,23081,23081,B5,G6350,S,7753698,14654483,+37.1935699,-076.7525080,"POLYGON ((-76.79774 37.21409, -76.79134 37.220...",23081
33788,23147,23147,B5,G6350,S,143646,0,+37.3696361,-077.0391136,"POLYGON ((-77.04292 37.36996, -77.04143 37.370...",23147
33789,23298,23298,B5,G6350,S,288664,0,+37.5417074,-077.4295300,"POLYGON ((-77.43285 37.54038, -77.43261 37.540...",23298


In [8]:
all_shp= zip_shp[zip_shp['ZIP'].isin(all_data['Zip Code'])]

In [9]:
all_shp

,ZCTA5CE20,GEOID20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry,ZIP
2209,03038,03038,B5,G6350,S,92840001,2947508,+42.8927614,-071.2767629,"POLYGON ((-71.35557 42.93741, -71.35543 42.937...",3038
2298,01845,01845,B5,G6350,S,68399834,3709019,+42.6739092,-071.0913339,"POLYGON ((-71.14586 42.69848, -71.14582 42.698...",1845
2299,02026,02026,B5,G6350,S,26600398,1072541,+42.2468719,-071.1794622,"POLYGON ((-71.23174 42.25928, -71.23171 42.259...",2026
2304,02368,02368,B5,G6350,S,24147379,1046980,+42.1789925,-071.0526540,"POLYGON ((-71.09099 42.18874, -71.09075 42.188...",2368
2396,01810,01810,B5,G6350,S,79050120,3442277,+42.6480437,-071.1617514,"POLYGON ((-71.25619 42.65714, -71.25559 42.657...",1810
...,...,...,...,...,...,...,...,...,...,...,...
32375,94903,94903,B5,G6350,S,51501250,1758406,+38.0221295,-122.5588100,"POLYGON ((-122.62460 38.03976, -122.62454 38.0...",94903
32377,94920,94920,B5,G6350,S,16861376,14740714,+37.8867683,-122.4626194,"POLYGON ((-122.50765 37.90464, -122.50687 37.9...",94920
32406,94947,94947,B5,G6350,S,56812613,513110,+38.1061995,-122.6249851,"POLYGON ((-122.70020 38.13417, -122.69990 38.1...",94947
32412,94960,94960,B5,G6350,S,15227188,0,+37.9970232,-122.5783384,"POLYGON ((-122.61316 38.01817, -122.61277 38.0...",94960


In [10]:
all_data['year'] =  pd.DatetimeIndex(all_data['y_dt']).year
all_data['quarter'] = pd.DatetimeIndex(all_data['y_dt']).quarter

In [11]:
all_data

,y_dt,Zip Code,Population Change (%),Housing Change (%),Housing/person Change (%),rent_change,zip,state,msa,year,quarter
0,2011-07-01,75001,0.060002,0.049557,-0.011414,0.055810,75001,48,Dallas,2011,3
1,2011-10-01,75001,0.060002,0.049557,-0.011200,0.065749,75001,48,Dallas,2011,4
2,2012-01-01,75001,0.060155,0.049682,-0.011021,0.108097,75001,48,Dallas,2012,1
3,2012-04-01,75001,0.053480,0.048808,-0.005096,0.050549,75001,48,Dallas,2012,2
4,2012-07-01,75001,0.047034,0.048059,0.000669,-0.006383,75001,48,Dallas,2012,3
...,...,...,...,...,...,...,...,...,...,...,...
13112,2018-10-01,3079,0.005065,0.012164,0.007165,0.201314,03079,33,Boston,2018,4
13113,2019-01-01,3079,0.005135,0.007970,0.002874,0.197204,03079,33,Boston,2019,1
13114,2019-04-01,3079,0.005100,0.003691,-0.001364,0.218893,03079,33,Boston,2019,2
13115,2019-07-01,3079,0.005135,-0.000671,-0.005717,0.216178,03079,33,Boston,2019,3


In [12]:
merged_geo_rent_chg = all_shp.merge(all_data[['msa','Zip Code', 'year', 'rent_change']], 
                                   how='left', 
                                   left_on='ZIP', 
                                   right_on='Zip Code')


In [13]:
merged_geo_rent_chg.isna().any()

ZCTA5CE20      False
GEOID20        False
CLASSFP20      False
MTFCC20        False
FUNCSTAT20     False
ALAND20        False
AWATER20       False
INTPTLAT20     False
INTPTLON20     False
geometry       False
ZIP            False
msa            False
Zip Code       False
year           False
rent_change    False
dtype: bool

In [14]:
merged_geo_rent_chg= merged_geo_rent_chg.drop(columns = ['ZCTA5CE20','GEOID20','CLASSFP20','MTFCC20','FUNCSTAT20','ALAND20','AWATER20', 'INTPTLAT20','INTPTLON20','Zip Code'])

In [15]:
merged_geo_rent_chg[['year', 'rent_change']] = merged_geo_rent_chg[['year', 'rent_change']].fillna(0)

In [16]:
merged_geo_rent_chg

,geometry,ZIP,msa,year,rent_change
0,"POLYGON ((-71.35557 42.93741, -71.35543 42.937...",3038,Boston,2011,0.044587
1,"POLYGON ((-71.35557 42.93741, -71.35543 42.937...",3038,Boston,2011,0.065199
2,"POLYGON ((-71.35557 42.93741, -71.35543 42.937...",3038,Boston,2012,0.050559
3,"POLYGON ((-71.35557 42.93741, -71.35543 42.937...",3038,Boston,2012,0.033276
4,"POLYGON ((-71.35557 42.93741, -71.35543 42.937...",3038,Boston,2012,0.040121
...,...,...,...,...,...
13112,"POLYGON ((-122.60492 37.87560, -122.60405 37.8...",94965,SF,2018,-0.004159
13113,"POLYGON ((-122.60492 37.87560, -122.60405 37.8...",94965,SF,2019,-0.112179
13114,"POLYGON ((-122.60492 37.87560, -122.60405 37.8...",94965,SF,2019,-0.037625
13115,"POLYGON ((-122.60492 37.87560, -122.60405 37.8...",94965,SF,2019,-0.024428


In [17]:
merged_geo_rent_chg.describe()

,ZIP,year,rent_change
count,13117.000000,13117.000000,13117.000000
mean,41393.262331,2015.222307,0.050891
std,33112.963093,2.461719,0.105945
min,1460.000000,2011.000000,-1.173879
25%,2421.000000,2013.000000,-0.000496
50%,30281.000000,2015.000000,0.047866
75%,75146.000000,2017.000000,0.099100
max,94965.000000,2019.000000,1.153854


In [18]:
def plot_enrollment1(df, name, year,msa):
    fig, ax = plt.subplots(figsize=(12,12))
    vmin = merged_geo_rent_chg['rent_change'].min()
    vmax = merged_geo_rent_chg['rent_change'].max()
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='2%', pad=0.5)
    cmap = plt.cm.get_cmap('RdYlBu')
    merged_geo_rent_chg[merged_geo_rent_chg['year'] == year][merged_geo_rent_chg['msa'] == msa][['geometry']].plot( 
        #column = 'rent_change',
        ax = ax, 
        color = 'white', 
        edgecolor = 'black', 
        cax = cax, 
        vmin = vmin,
        vmax = vmax, 
        legend = True );
    
    if name == 'all':
        merged_geo_rent_chg[merged_geo_rent_chg['year'] == year][merged_geo_rent_chg['msa'] == msa][['geometry', 'rent_change']].plot( 
            column='rent_change',
            ax=ax, 
            edgecolor='black', 
            cmap=cmap, 
            cax=cax, 
            vmin=vmin,
            vmax=vmax, 
            legend=True 
        );
    else:
        merged_geo_rent_chg[merged_geo_rent_chg['msa'] == msa][(merged_geo_rent_chg['year'] == year) & (merged_geo_rent_chg['ZIP'] == name)][['geometry', 'rent_change']].plot( 
            column='rent_change',
            ax=ax, 
            edgecolor='black', 
            cmap=cmap, 
            cax=cax, 
            vmin=vmin,
            vmax=vmax, 
            legend=True 
        );
            #cbar = True)
    
    ax.axis('off')
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=matplotlib.colors.Normalize(vmin=vmin, vmax=vmax))

    plt.colorbar(sm, cax=cax)


country_names = merged_geo_rent_chg['ZIP'].unique()
country_names = list(country_names)
country_names.insert(0, 'all')
msa = merged_geo_rent_chg['msa'].unique()
years = merged_geo_rent_chg['year'].unique().astype(int)

@interact(zip_code = country_names, year = years, msa = msa)
def make_plot_enrollment1(zip_code=country_names, year = years, msa= msa):
    plot_enrollment1(merged_geo_rent_chg,zip_code, year, msa)

    

interactive(children=(Dropdown(description='zip_code', options=('all', 3038, 1845, 2026, 2368, 1810, 1876, 190…

In [ ]:
merged_geo_rent_chg.isna().any()